In [1]:
import tensorflow as tf
from tensorflow import keras

import mlflow
import mlflow.keras

print('TensorFlow version: {}'.format(tf.__version__))

TensorFlow version: 2.9.1


In [2]:
mlflow.set_tracking_uri("http://3.35.212.118:5000")
mlflow.set_experiment("tf-fashion-mnist-experiment")

2022/08/15 18:26:02 INFO mlflow.tracking.fluent: Experiment with name 'tf-fashion-mnist-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/2', experiment_id='2', lifecycle_stage='active', name='tf-fashion-mnist-experiment', tags={}>

In [3]:
mlflow.tensorflow.autolog()
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# scale the values to 0.0 to 1.0
train_images = train_images / 255.0
test_images = test_images / 255.0

# reshape for feeding into the model
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

print('\ntrain_images.shape: {}, of {}'.format(train_images.shape, train_images.dtype))
print('test_images.shape: {}, of {}'.format(test_images.shape, test_images.dtype))

model = keras.Sequential([
  keras.layers.Conv2D(input_shape=(28,28,1), filters=8, kernel_size=3, 
                      strides=2, activation='relu', name='Conv1'),
  keras.layers.Flatten(),
  keras.layers.Dense(10, name='Dense')
])
model.summary()

testing = False
epochs = 5

model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])
model.fit(train_images, train_labels, epochs=epochs)

test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test loss: {}'.format(test_loss))
print('Test accuracy: {}'.format(test_acc))


train_images.shape: (60000, 28, 28, 1), of float64
test_images.shape: (10000, 28, 28, 1), of float64
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv1 (Conv2D)              (None, 13, 13, 8)         80        
                                                                 
 flatten (Flatten)           (None, 1352)              0         
                                                                 
 Dense (Dense)               (None, 10)                13530     
                                                                 
Total params: 13,610
Trainable params: 13,610
Non-trainable params: 0
_________________________________________________________________


2022-08-15 18:26:21.326297: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022/08/15 18:26:21 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '17ea5b3468b548a7bc599837b8f2ab5a', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.5483 - sparse_categorical_accuracy: 0.8087
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4255 - sparse_categorical_accuracy: 0.8522
Epoch 3/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3929 - sparse_categorical_accuracy: 0.8628
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3722 - sparse_categorical_accuracy: 0.8704
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3534 - sparse_categorical_accuracy: 0.8754


INFO:tensorflow:Assets written to: /var/folders/t5/j16x5yxj6yb2f6tzgwvt3jk40000gn/T/tmpzq1cwivy/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/t5/j16x5yxj6yb2f6tzgwvt3jk40000gn/T/tmpzq1cwivy/model/data/model/assets
2022/08/15 18:26:56 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/mzc01-lsi/anaconda3/envs/qraft-mlflow-env/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


313/313 [==============================] - 1s 1ms/step - loss: 0.3830 - sparse_categorical_accuracy: 0.8641
Test loss: 0.38299262523651123
Test accuracy: 0.8640999794006348


In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# scale the values to 0.0 to 1.0
train_images = train_images / 255.0
test_images = test_images / 255.0

# reshape for feeding into the model
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

print('\ntrain_images.shape: {}, of {}'.format(train_images.shape, train_images.dtype))
print('test_images.shape: {}, of {}'.format(test_images.shape, test_images.dtype))


train_images.shape: (60000, 28, 28, 1), of float64
test_images.shape: (10000, 28, 28, 1), of float64


In [3]:
model = keras.Sequential([
  keras.layers.Conv2D(input_shape=(28,28,1), filters=8, kernel_size=3, 
                      strides=2, activation='relu', name='Conv1'),
  keras.layers.Flatten(),
  keras.layers.Dense(10, name='Dense')
])
model.summary()

testing = False
epochs = 5

model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv1 (Conv2D)              (None, 13, 13, 8)         80        
                                                                 
 flatten (Flatten)           (None, 1352)              0         
                                                                 
 Dense (Dense)               (None, 10)                13530     
                                                                 
Total params: 13,610
Trainable params: 13,610
Non-trainable params: 0
_________________________________________________________________


2022-08-15 17:54:36.343898: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
model.fit(train_images, train_labels, epochs=epochs)

test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test loss: {}'.format(test_loss))
print('Test accuracy: {}'.format(test_acc))

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.5646 - sparse_categorical_accuracy: 0.8012
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4358 - sparse_categorical_accuracy: 0.8472
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3833 - sparse_categorical_accuracy: 0.8655
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3544 - sparse_categorical_accuracy: 0.8741
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.3640 - sparse_categorical_accuracy: 0.8693
Test loss: 0.3639596104621887
Test accuracy: 0.8693000078201294


In [4]:
import bentoml

bentoml.tensorflow.save_model('tf-fashion-mnist', model)

/Users/mzc01-lsi/anaconda3/envs/qraft-mlflow-env/lib/python3.10/site-packages/bentoml/_internal/utils/tensorflow.py:297: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  function_map = {k: getattr(m, k, None) for k in dir(m)}
/Users/mzc01-lsi/anaconda3/envs/qraft-mlflow-env/lib/python3.10/site-packages/bentoml/_internal/utils/tensorflow.py:297: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  function_map = {k: getattr(m, k, None) for k in dir(m)}


INFO:tensorflow:Assets written to: /var/folders/t5/j16x5yxj6yb2f6tzgwvt3jk40000gn/T/tmptlibd939bentoml_model_tf-fashion-mnist/assets


INFO:tensorflow:Assets written to: /var/folders/t5/j16x5yxj6yb2f6tzgwvt3jk40000gn/T/tmptlibd939bentoml_model_tf-fashion-mnist/assets


Model(tag="tf-fashion-mnist:rs3njyq4psfsvlg6", path="/Users/mzc01-lsi/bentoml/models/tf-fashion-mnist/rs3njyq4psfsvlg6/")